In [37]:
from google.cloud import bigquery
from google.oauth2 import service_account

In [51]:
# Construct a BigQuery client object.
client = bigquery.Client(project='mmtc-staging')

# TODO(developer): Set table_id to the ID of the table
#                  to add an empty column.
#table_id = "mmtc-staging.analytics.latest_dataset"

#table = client.get_table(table_id)  # Make an API request.

#original_schema = table.schema
#new_schema = original_schema[:]  # Creates a copy of the schema.
#new_schema.append(bigquery.SchemaField("FundingSource", "STRING"))
#new_schema.append(bigquery.SchemaField("AppraisedValue", "FLOAT"))
#new_schema.append(bigquery.SchemaField("TimeWindow", "INTEGER"))

#table.schema = new_schema
#table = client.update_table(table, ["schema"])  # Make an API request.


# perform a query
query = """
SELECT FundingSource, LTV, State, LoanProcessor,LoanOfficer, LoanAmount, 
       IF((ABS(TimeWindow)>1000),0,TimeWindow) AS TimeWindow, 
       AppraisedValue
FROM  
  (SELECT FundingSource, LTV, State, LoanProcessor,LoanOfficer, LoanAmount, AppraisedValue, 
          (DATETIME_DIFF(parse_datetime('%m/%d/%Y',CloseDate), parse_datetime('%m/%d/%Y',SignedDate), DAY)) AS TimeWindow    
   FROM
        (SELECT  
            FundingSource, 
            SubjectLoan_SelfReportedARLTVRatioPercent.float AS LTV,
            SubjectProperty_StateCode AS State,
            SubjectLoan_BorrowerRequestedLoanAmount AS LoanAmount,
            AppraisedValue,
            REPLACE(LenderRepresentative_LenderRepresentativeSignatureName,' ','') AS LoanProcessor,
            REPLACE(LoanOfficer_FullName,' ','') AS LoanOfficer,
            IF(REGEXP_CONTAINS(SubjectLoan_LoanEstimatedClosingDate, '^[0-9]'), 
                SubjectLoan_LoanEstimatedClosingDate, "01/01/2000") AS CloseDate,
            IF(REGEXP_CONTAINS(Borrower_BorrowerApplicationSignedDate, '^[0-9]'), 
                Borrower_BorrowerApplicationSignedDate, "01/01/2000") AS SignedDate
        FROM
          `mmtc-staging.analytics.latest_dataset_724`) AS Table_1)
"""
query_job = client.query(query)  # Make an API request.
df = query_job.to_dataframe()
df

,FundingSource,LTV,State,LoanProcessor,LoanOfficer,LoanAmount,TimeWindow,AppraisedValue
0,None,NaN,None,None,MeganTrammel,420000,0,NaN
1,None,NaN,None,None,AaronPfeffer,300000,0,NaN
2,None,NaN,AZ,None,HerbBourdeaux,0,0,NaN
3,None,NaN,None,None,None,0,0,NaN
4,None,NaN,None,None,SusanAubin,0,0,NaN
5,None,NaN,None,None,BobThomas,0,0,NaN
6,None,NaN,None,None,None,0,0,NaN
7,None,NaN,CA,None,AaronPfeffer,675000,0,NaN
8,None,NaN,CA,None,AaronPfeffer,350000,0,NaN
9,None,NaN,CA,None,TravisMullen,436500,0,NaN


In [52]:
print (df.dtypes) 

FundingSource      object
LTV               float64
State              object
LoanProcessor      object
LoanOfficer        object
LoanAmount          int64
TimeWindow          int64
AppraisedValue    float64
dtype: object


In [49]:
df['Start_date'] = pd.to_datetime(df['Start_date'].str.strip(), format='%d/%m/%Y')

ValueError: time data '07/13/2020' does not match format '%d/%m/%Y' (match)

In [28]:
query_job

In [8]:
import pandas as pd

In [9]:
df = pd.read_json('data.json')

In [17]:
rloan = df.T
rloan

,SubjectLoan_LoanProcessType,LoanDeferment_DefermentFeeBasisPointAmount,SubjectProperty_FinancedUnitCount,SubjectLoan_PrepaymentPenaltyIndicator,Borrower_AlimonyChildSupportObligationIndicator,LoanDeferment_DefermentMonthCount,CrossCollateral_PropertyValuationAmount,SubjectProperty_DSCRPercent,SubjectLoan_RepeatCustomerLoanCount,BorrowerEmployment,...,Borrower_IntentToOccupyType,Borrower_PriorContractExperienceIndicator,Borrower_HomeownerPastThreeYearsType,SubjectLoan_RehabilitationEstimatedPropertyValueAmount,Borrower_HMDARaceType,Borrower_HMDAEthnicityOriginType,Borrower_HMDARaceDesignationType,Borrower_OwnershipInterestPastThreeYearsTitleType,Borrower_PriorPropertyUsageType,SubjectProperty_PropertyDwellingUnitLeaseProvidedIndicator
03d09e77-e7d8-45d9-9250-1e93fff9f7ce,HavePropertyUnderContract,33,0,False,False,7,0,0,0,"[{'EmploymentMonthlyIncomeAmount': 5000, 'Empl...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10984a28-2f06-48be-bf34-e84f299122ad,HavePropertyUnderContract,33,0,False,False,7,0,0,2,"[{'EmploymentMonthlyIncomeAmount': 5000, 'Empl...",...,False,True,False,0,NaN,NaN,NaN,NaN,NaN,NaN
1778808f-33e7-4f66-b02d-f051b0d720f8,ActivelyLookingForAProperty,0,0,False,NaN,0,0,0,0,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
24c0ceef-ac92-4fcc-86ec-20a643f3f4b4,HavePropertyUnderContract,0,0,False,NaN,0,0,0,0,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
29a60b1e-0540-4a43-8365-40d496eace1f,IdentifiedAProperty,33,0,False,NaN,10,0,0,5,NaN,...,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34278163-b794-442a-8a5e-01bde12d92f6,HavePropertyUnderContract,33,0,False,False,7,0,0,4,"[{'EmploymentMonthlyIncomeAmount': 10333, 'Emp...",...,False,False,False,850000,[White],NaN,NaN,NaN,NaN,NaN
3e06d1d9-7080-4a2b-bcde-0731f71c955a,HavePropertyUnderContract,2,0,False,NaN,10,0,0,0,NaN,...,NaN,NaN,NaN,500000,NaN,NaN,NaN,NaN,NaN,NaN
4b46ae62-5a0c-4a56-96e5-0ccf88029810,IOwnThePropertyOrLand,0,0,False,NaN,0,0,0,3,NaN,...,NaN,False,NaN,0,NaN,[Mexican],NaN,NaN,NaN,NaN
4df33c06-ded2-42ce-a937-0358aa4db42e,NaN,0,0,NaN,NaN,0,0,0,0,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
504ac667-fe6b-4d94-b89e-01b3724e8ed1,HavePropertyUnderContract,0,0,True,NaN,0,0,0,0,NaN,...,NaN,True,False,240000,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
rloan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, 03d09e77-e7d8-45d9-9250-1e93fff9f7ce to fefd5039-5496-40e6-b575-b9818020ac96
Columns: 140 entries, SubjectLoan_LoanProcessType to SubjectProperty_PropertyDwellingUnitLeaseProvidedIndicator
dtypes: object(140)
memory usage: 47.5+ KB


In [15]:
rloan.describe()

,SubjectLoan_LoanProcessType,LoanDeferment_DefermentFeeBasisPointAmount,SubjectProperty_FinancedUnitCount,SubjectLoan_PrepaymentPenaltyIndicator,Borrower_AlimonyChildSupportObligationIndicator,LoanDeferment_DefermentMonthCount,CrossCollateral_PropertyValuationAmount,SubjectProperty_DSCRPercent,SubjectLoan_RepeatCustomerLoanCount,BorrowerEmployment,...,Borrower_IntentToOccupyType,Borrower_PriorContractExperienceIndicator,Borrower_HomeownerPastThreeYearsType,SubjectLoan_RehabilitationEstimatedPropertyValueAmount,Borrower_HMDARaceType,Borrower_HMDAEthnicityOriginType,Borrower_HMDARaceDesignationType,Borrower_OwnershipInterestPastThreeYearsTitleType,Borrower_PriorPropertyUsageType,SubjectProperty_PropertyDwellingUnitLeaseProvidedIndicator
count,38,42,42,37,18,42,42,42,42,23,...,19,29,19,39,11,4,2,5,5,1
unique,4,4,1,2,1,4,1,1,9,22,...,1,2,2,12,3,2,2,3,2,1
top,HavePropertyUnderContract,33,0,False,False,7,0,0,0,"[{'EmploymentMonthlyIncomeAmount': 5000, 'Empl...",...,False,False,False,0,[White],[Mexican],[OtherAsian],[JointlyWithYourSpouse],[PrincipalResidence],False
freq,22,22,42,32,18,19,42,42,33,2,...,19,18,10,27,7,3,1,3,4,1


In [20]:
rloan.isna().sum()

SubjectLoan_LoanProcessType                                    4
LoanDeferment_DefermentFeeBasisPointAmount                     0
SubjectProperty_FinancedUnitCount                              0
SubjectLoan_PrepaymentPenaltyIndicator                         5
Borrower_AlimonyChildSupportObligationIndicator               24
                                                              ..
Borrower_HMDAEthnicityOriginType                              38
Borrower_HMDARaceDesignationType                              40
Borrower_OwnershipInterestPastThreeYearsTitleType             37
Borrower_PriorPropertyUsageType                               37
SubjectProperty_PropertyDwellingUnitLeaseProvidedIndicator    41
Length: 140, dtype: int64

In [21]:
rloan.to_csv('real_loan.csv')

In [22]:
list(rloan.columns)

['SubjectLoan_LoanProcessType',
 'LoanDeferment_DefermentFeeBasisPointAmount',
 'SubjectProperty_FinancedUnitCount',
 'SubjectLoan_PrepaymentPenaltyIndicator',
 'Borrower_AlimonyChildSupportObligationIndicator',
 'LoanDeferment_DefermentMonthCount',
 'CrossCollateral_PropertyValuationAmount',
 'SubjectProperty_DSCRPercent',
 'SubjectLoan_RepeatCustomerLoanCount',
 'BorrowerEmployment',
 'SubjectProperty_PropertyAreaIncreaseIndicator',
 'LoanRequestStrengths_LoanRequestStrengthDescription',
 'SubjectLoan_RequiredARLTVRatioPercent',
 'SubjectProperty_StructuralChangesIndicator',
 'SubjectLoan_PropertyTaxLandValueAmount',
 'Lender_RequestingPartyBranchIdentifier',
 'LoanRequestWeaknesses_LoanRequestWeaknessDescription',
 'SubjectProperty_TotalMonthlyHousingExpenseAmount',
 'SubjectLoan_NoteRatePercent',
 'SubjectLoan_BorrowerRequestedLoanAmount',
 'SubjectProperty_StateName',
 'SubjectLoan_SelfReportedARLTVRatioPercent',
 'SubjectLoan_InterestPaymentAmount',
 'SubjectLoan_RequiredProperty

In [23]:
print (rloan.dtypes) 

SubjectLoan_LoanProcessType                                   object
LoanDeferment_DefermentFeeBasisPointAmount                    object
SubjectProperty_FinancedUnitCount                             object
SubjectLoan_PrepaymentPenaltyIndicator                        object
Borrower_AlimonyChildSupportObligationIndicator               object
                                                               ...  
Borrower_HMDAEthnicityOriginType                              object
Borrower_HMDARaceDesignationType                              object
Borrower_OwnershipInterestPastThreeYearsTitleType             object
Borrower_PriorPropertyUsageType                               object
SubjectProperty_PropertyDwellingUnitLeaseProvidedIndicator    object
Length: 140, dtype: object
